## Sandbox

Un  `.ipynb` pour tester des idées :)

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));
include(joinpath("src", "dual_solve.jl"));

  Activating project at `c:\cours_MASTER\PROJ\workspace\PROJ`


In [2]:
# file = joinpath("data", "20_USA-road-d.BAY.gr")
file = joinpath("data/instances", "20_USA-road-d.BAY.gr")
infos = open(
    readInstance,
    file,
);

In [3]:
graph = graphFromData(infos);

In [4]:
function flow_creation(i::Int64)::Int64
    return Int64(graph[i].is_s) - Int64(graph[i].is_t)
end

flow_creation (generic function with 1 method)

In [5]:
#declaration du modele
m = Model(Gurobi.Optimizer)
m = Model(
                optimizer_with_attributes(
                Gurobi.Optimizer, "Presolve" => 0
                ))
set_silent(m)
#declaration des variables
@variable(m, a[edge_labels(graph)], Bin)  # vaut 1 ssi arc (i,j) choisi
@objective(m, Min, sum(graph[i, j].d * a[(i, j)] for (i, j) in edge_labels(graph)))
# declaration des contraintes
# Conservation du flot :
@constraint(
m,
flow_cons[i in labels(graph)],
(
    sum(a[(i, j)] for j in outneighbor_labels(graph, i))
    -
    sum(a[(j, i)] for j in inneighbor_labels(graph, i))
    == flow_creation(i)
))
@constraint(m, sum(graph[i].p * a[(i, j)] for (i, j) in edge_labels(graph))
+ graph[graph[].t].p <= graph[].big_s )

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-25
Set parameter Presolve to value 0


14 a[(1, 2)] + 14 a[(1, 3)] + 14 a[(1, 4)] + 14 a[(1, 5)] + 14 a[(1, 6)] + 14 a[(1, 7)] + 14 a[(1, 8)] + 14 a[(1, 11)] + 14 a[(1, 12)] + 14 a[(1, 14)] + 14 a[(1, 18)] + 14 a[(1, 19)] + 14 a[(1, 20)] + 4 a[(2, 1)] + 4 a[(2, 9)] + 4 a[(2, 13)] + 7 a[(3, 1)] + 7 a[(3, 4)] + 7 a[(3, 9)] + 7 a[(3, 11)] + 7 a[(3, 13)] + 7 a[(3, 19)] + 9 a[(4, 1)] + 9 a[(4, 3)] + 9 a[(4, 9)] + 9 a[(4, 10)] + 9 a[(4, 13)] + 9 a[(4, 14)] + 9 a[(4, 16)] + 9 a[(5, 1)] + [[...76 terms omitted...]] + 7 a[(15, 11)] + 7 a[(15, 19)] + 7 a[(16, 4)] + 7 a[(16, 5)] + 7 a[(16, 11)] + 7 a[(16, 12)] + 7 a[(16, 19)] + 7 a[(16, 20)] + 8 a[(18, 1)] + 8 a[(18, 5)] + 8 a[(18, 7)] + 8 a[(18, 9)] + 8 a[(18, 12)] + 8 a[(18, 13)] + 8 a[(18, 20)] + 12 a[(19, 1)] + 12 a[(19, 3)] + 12 a[(19, 9)] + 12 a[(19, 10)] + 12 a[(19, 13)] + 12 a[(19, 14)] + 12 a[(19, 16)] + 9 a[(20, 1)] + 9 a[(20, 6)] + 9 a[(20, 9)] + 9 a[(20, 10)] + 9 a[(20, 13)] + 9 a[(20, 16)] + 9 a[(20, 17)] + 9 a[(20, 18)] <= 59

In [9]:
function resolution(
    graph::MetaGraph;
    timelimit::Float64=3600.0
)::ModResultWrapper
    start = time()
    s::Int64 = graph[].s
    t::Int64 = graph[].t
    JuMP.optimize!(m)
    m_value = JuMP.objective_value(m)
    A = JuMP.value.(a)
    for (i, j) in edge_labels(graph)
        if A[(i,j)] > 0
            println("(",i,",",j,") = ", A[(i,j)])
        end
    end
    println("Objective value m: ", m_value)
    println("elapsed time : ", time() - start)
    return (
        primal_status=primal_status(m),
        dual_status=dual_status(m),
        term_status=termination_status(m),
        obj_value=objective_value(m),
        lower_bound=objective_bound(m),
        upper_bound=(primal_status(m) == FEASIBLE_POINT ? objective_value(m) : Inf),
        a=Dict{Tuple{Int64, Int64}, Float64}((i, j) => value(a[(i, j)]) for (i, j) in edge_labels(graph)),
    )
end

resolution (generic function with 1 method)

In [11]:
res = resolution(graph; timelimit=time() + 60.0)
println(mkPath(res.a))
println(res.obj_value)

(1,20) = 1.0
(15,19) = 1.0
(19,1) = 1.0
(20,17) = 1.0
Objective value m: 9365.0
elapsed time : 0.006000041961669922
[15, 19, 1, 20, 17]
9365.0
